In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import (ColumnDataSource, Range1d, BoxAnnotation, 
                          Label, Arrow, NormalHead)
from bokeh.io import output_notebook

# Determine where to show (notebook or browser)
# output_notebook()  # Uncomment if using Jupyter

# Colors
COLOR_AUDIO = '#EF553B'      # Red
COLOR_SYMPTOMS = '#7f7f7f'   # Grey
COLOR_COMBINED = '#636EFA'   # Blue
COLOR_UTILITY = '#00CC96'    # Green

# Data generation
np.random.seed(42)
num_samples = 2000
num_bootstraps = 100

# True labels (0 = negative, 1 = positive)
labels_true = np.concatenate([np.zeros(num_samples), np.ones(num_samples)])

# Generate synthetic scores
base_signal = np.concatenate([
    np.random.normal(0, 1, num_samples),
    np.random.normal(1.2, 1, num_samples)
])
noise_audio = np.random.normal(0, 1.2, 2 * num_samples)
noise_symptoms = np.random.normal(0, 0.8, 2 * num_samples)

scores_audio = 0.4 * base_signal + 0.6 * noise_audio
scores_symptoms = 0.7 * base_signal + 0.3 * noise_symptoms
scores_combined = scores_symptoms + 0.1 * scores_audio

# Store model data
models = {
    'Audio': {
        'scores': scores_audio,
        'color': COLOR_AUDIO,
        'label': 'Audio only'
    },
    'Symptoms': {
        'scores': scores_symptoms,
        'color': COLOR_SYMPTOMS,
        'label': 'Symptoms only'
    },
    'Combined': {
        'scores': scores_combined,
        'color': COLOR_COMBINED,
        'label': 'Combined'
    }
}

# Bootstrap calculations
fpr_baseline = np.linspace(0, 1, 101)

for model_name, model_data in models.items():
    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(labels_true, model_data['scores'])
    model_data['fpr'] = fpr
    model_data['tpr'] = tpr
    model_data['auc'] = auc(fpr, tpr)
    model_data['specificity'] = 1 - fpr
    
    # Bootstrap confidence intervals
    tpr_bootstrap = []
    for _ in range(num_bootstraps):
        indices = np.random.randint(0, len(labels_true), len(labels_true))
        if len(np.unique(labels_true[indices])) < 2:
            continue
        
        fpr_boot, tpr_boot, _ = roc_curve(labels_true[indices], model_data['scores'][indices])
        tpr_bootstrap.append(np.interp(fpr_baseline, fpr_boot, tpr_boot))
    
    tpr_bootstrap = np.array(tpr_bootstrap)
    model_data['tpr_mean'] = np.mean(tpr_bootstrap, axis=0)
    model_data['tpr_lower'] = np.percentile(tpr_bootstrap, 2.5, axis=0)
    model_data['tpr_upper'] = np.percentile(tpr_bootstrap, 97.5, axis=0)

# Plot setup
PLOT_TOOLS = "pan,wheel_zoom,reset,save"

# LEFT PLOT: Original style (reversed specificity axis)
plot_original = figure(
    title="Original style (Visual inflation)",
    x_axis_label="Specificity (reversed axis)",
    y_axis_label="Sensitivity",
    height=500, width=600,
    tools=PLOT_TOOLS,
    x_range=Range1d(1.05, -0.05),  # Reversed
    y_range=Range1d(-0.05, 1.05)
)

# Highlight useless region (specificity < 50%)
useless_region = BoxAnnotation(left=0.5, right=-0.05, fill_color='gray', fill_alpha=0.1)
plot_original.add_layout(useless_region)

label_useless = Label(
    x=0.25, y=0.5,
    text="Operationally useless region\n(Specificity < 50%)",
    text_align='center', text_baseline='middle',
    angle=1.5707,  # 90 degrees
    text_font_size='12px', text_color='#444444',
    background_fill_color='white', background_fill_alpha=0.6
)
plot_original.add_layout(label_useless)

# Plot curves with error bars at specific points
specificity_targets = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

for model_name, model_data in models.items():
    # Main curve
    plot_original.line(
        model_data['specificity'], model_data['tpr'],
        color=model_data['color'], line_width=2,
        legend_label=f"{model_data['label']} (AUC={model_data['auc']:.2f})"
    )
    
    # Error bars at specific specificities
    seg_x, seg_y_lower, seg_y_upper = [], [], []
    for target_spec in specificity_targets:
        idx = np.abs(model_data['specificity'] - target_spec).argmin()
        x_value = model_data['specificity'][idx]
        y_value = model_data['tpr'][idx]
        seg_x.append(x_value)
        seg_y_lower.append(y_value - 0.04)
        seg_y_upper.append(y_value + 0.04)
    
    plot_original.segment(
        x0=seg_x, y0=seg_y_lower, x1=seg_x, y1=seg_y_upper,
        color=model_data['color'], line_width=1, line_alpha=0.6
    )

plot_original.legend.location = "bottom_right"
plot_original.legend.click_policy = "hide"
plot_original.legend.background_fill_alpha = 0.0
plot_original.grid.visible = False

# RIGHT PLOT: Nature style (focused on high specificity)
plot_focused = figure(
    title="Focused on clinical utility",
    x_axis_label="False positive rate (FPR = 1 - Specificity)",
    y_axis_label="True positive rate (sensitivity)",
    height=500, width=600,
    tools=PLOT_TOOLS,
    x_range=Range1d(0.0, 0.4),
    y_range=Range1d(0.0, 1.02)
)

# Highlight clinical utility zone
utility_zone = BoxAnnotation(left=-0.01, right=0.2, fill_color=COLOR_UTILITY, fill_alpha=0.08)
plot_focused.add_layout(utility_zone)

label_utility = Label(
    x=0.02, y=0.96,
    text="Clinical utility zone\n(High specificity / Low FPR)",
    text_align='left', text_baseline='top',
    text_color='#005500', text_font_size='11px'
)
plot_focused.add_layout(label_utility)

# Plot curves with confidence intervals
for model_name, model_data in models.items():
    # Confidence interval ribbons
    plot_focused.varea(
        x=fpr_baseline,
        y1=model_data['tpr_lower'],
        y2=model_data['tpr_upper'],
        fill_color=model_data['color'], fill_alpha=0.15
    )
    
    # Mean curves
    plot_focused.line(
        fpr_baseline, model_data['tpr_mean'],
        color=model_data['color'], line_width=2.5,
        legend_label=model_data['label']
    )

# Annotate operating point (95% specificity = 5% FPR)
operating_fpr = 0.05
operating_idx = np.abs(fpr_baseline - operating_fpr).argmin()
operating_sensitivity = models['Combined']['tpr_mean'][operating_idx]

# Mark the point
plot_focused.circle(
    x=[operating_fpr], y=[operating_sensitivity],
    size=8, color='black', line_color='white', line_width=1
)

# Add annotation with arrow
annotation_text = Label(
    x=0.15, y=0.35,
    text=f"Real-world operating point\nFPR: 5% (Specificity: 95%)\nSensitivity: {operating_sensitivity:.0%}",
    text_font_size="11px",
    background_fill_color="white", background_fill_alpha=0.9,
    border_line_color="#cccccc"
)
plot_focused.add_layout(annotation_text)

annotation_arrow = Arrow(
    end=NormalHead(fill_color="black", size=10),
    x_start=0.15, y_start=0.35,
    x_end=operating_fpr, y_end=operating_sensitivity,
    line_color="black", line_width=0.8
)
plot_focused.add_layout(annotation_arrow)

# Style for Nature look
plot_focused.legend.location = "bottom_right"
plot_focused.legend.background_fill_alpha = 0.0
plot_focused.legend.border_line_alpha = 0.0
plot_focused.outline_line_color = None
plot_focused.axis.axis_line_color = "black"
plot_focused.grid.grid_line_color = "#dddddd"
plot_focused.grid.grid_line_dash = "dotted"

# Display side by side
layout = gridplot([[plot_original, plot_focused]])
show(layout)

In [ ]:
# %pip install bokeh